In [1]:
import json
import nltk
import os
import time
import copy
import sys
import re
import torch
import datetime
import torchtext
import pickle
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
from torchtext import data
from torchtext import datasets
from collections import Counter
from torchtext.vocab import GloVe
from nltk.tokenize.treebank import TreebankWordTokenizer
from nltk.tokenize.stanford import StanfordTokenizer

np.random.seed(15)
nltk_tokenizer = TreebankWordTokenizer()
ptb_tokenizer = StanfordTokenizer('../../../../data/stanford-parser-full-2018-02-27/stanford-parser.jar')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/Users/tusjoshi/Desktop/practice_projects/venv/lib/python3.7/site-packages/ipykernel_launcher.py:26: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.parse.corenlp.CoreNLPParser instead.'


In [2]:
def post_ptbtokenizer(text):
    return text.split(' ')

def ptbtokenizer(text, context=False):
    text = text.replace('’', "'")
    text = text.replace("..", ".")
    text = text.replace(". .", ".")
    text = text.replace("´", "'")
    text = text.replace("`", "'")
    doc = ptb_tokenizer.tokenize(text)
    
    output = []
    for token in doc:
        _token = re.split('([$.])', token)
        _token = [i for i in _token if len(i)>0]
        output = output + _token
    
    if context:
        output.append('<eoc>')
        
    return output

In [3]:
def create_checkpoint_paths(folder_path):
    if not os.path.isdir(folder_path):
        os.mkdir(folder_path)

In [4]:
class LoadData():
    def __init__(self, 
                 data_path, 
                 checkpoint_path):
        self.data_path = data_path
        self.checkpoint_path = checkpoint_path
        self.train_data = self.load_train_data()
        self.dev_data = self.load_dev_data()   
        
    def load_train_data(self):
        with open(self.data_path + 'train-v1.1.json', 'r') as f:
            train_data = json.load(f)
        print (f'\nFlattening SQUAD {train_data["version"]} - Train')
        train_data_flat, errors = self.load_squad_data(train_data)
        print (f'\nErronous Train Datapoints: {errors}')
        pd.DataFrame(train_data_flat).to_csv(self.checkpoint_path + 'train.csv', encoding='utf-8')
        
    def load_dev_data(self):
        with open(self.data_path + 'dev-v1.1.json', 'r') as f:
            dev_data = json.load(f)
        print (f'\nFlattening SQUAD Dev')
        dev_data_flat, errors = self.load_squad_data(dev_data)
        print (f'\nErronous Dev Datapoints: {errors}')
        pd.DataFrame(dev_data_flat).to_csv(self.checkpoint_path + 'dev.csv', encoding='utf-8')       
    
    def convert_charidx_to_wordidx(self, context_tok, ans_tok):                               
        length = len(ans_tok)
        _start, _end = None, None
        
        for i in range(len(context_tok)):
            context_text = " ".join(context_tok[i:i+length]).strip('.')
            ans_text = " ".join(ans_tok).strip('.')
            _ans_text = "".join(ans_tok).strip('.')
            __ans_text = " ".join(ans_tok).strip(',')
            
            if ans_text in context_text or _ans_text in context_text or __ans_text in context_text:
                _start = i
                _end = i+length
                break
          
        if _start == None:
            for i in range(len(context_tok)):
                if ans_tok[0] == context_tok[i]:
                    _start = i
                    _end = i+length
                    
        if _start == None:
            print (ans_tok)
            print (context_tok)
        
        if _end != None and _end >= len(context_tok):
            _end = len(context_tok) - 1
            
        return _start, _end
        
    def load_squad_data(self, data):
        progress = 0
        errors = 0
        start_time = time.time()
        flatened_data = []
        for topics in data['data']:
            title = topics['title']
            for paragraphs in topics['paragraphs']:
                context = paragraphs['context']
#                 context_tok = ptbtokenizer(context, context=True)
                for qas in paragraphs['qas']:
                    id = qas['id']
                    question = qas['question']
                    for answer in qas['answers']:
                        progress += 1
                        
                        time_delta = datetime.timedelta(seconds=np.round(time.time() - start_time, 0))
                        sys.stdout.write(f'\rCompleted: {progress} | Time: {time_delta}')
                        if progress > 2000:
                            continue
                        context_tok = ptbtokenizer(context, context=True)
                        
                        answer_start = answer['answer_start']
                        answer_end = answer['answer_start'] + len(answer['text'])
                        
                        ans_tok = ptbtokenizer(context[answer_start:answer_end])
                        question_tok = ptbtokenizer(question)
                        
                        _start, _end = self.convert_charidx_to_wordidx(context_tok, ans_tok)
                        if _start == None:
                            errors += 1
                            continue
                        flatened_data.append({'id': id,
                                              'context': context,
                                              'context_ptb_tok': ' '.join(context_tok),
                                              'question': question,
                                              'question_ptb_tok': ' '.join(question_tok),
                                              'answer_ptb_tok': ' '.join(ans_tok),
                                              'start_idx': _start,
                                              'end_idx': _end})
        
        return flatened_data, errors

In [5]:
class PreprocessData:
    def __init__(self, 
                 data_path, 
                 glove_size,
                 batch_size):
        device = torch.device("cuda:{}".format(args.gpu) if torch.cuda.is_available() else "cpu")
        
        # Defining the Fields
        self.RAW = data.RawField(is_target=False)
        self.WORDS = data.Field(batch_first=True, 
                           tokenize=post_ptbtokenizer, 
                           lower=True, 
                           include_lengths=True)
        self.CHAR = data.NestedField(data.Field(batch_first=True, 
                                           tokenize=list, 
                                           lower=True), 
                                tokenize=post_ptbtokenizer)
        
        self.INDEX = data.Field(sequential=False, 
                                unk_token=None, 
                                use_vocab=False)
        
        fields = {
            'id': ('id', self.RAW),
            'context_ptb_tok': [('context_words', self.WORDS), ('context_char', self.CHAR)],
            'question_ptb_tok': [('question_words', self.WORDS), ('question_char', self.CHAR)],
            'answer_ptb_tok': [('answer_words', self.WORDS), ('answer_char', self.CHAR)],
            'start_idx': ('start_idx', self.INDEX),
            'end_idx': ('end_idx', self.INDEX)
        }


        print ('Loading CSV Data Into Torch Tabular Dataset')
        self.train, self.dev = data.TabularDataset.splits(
            path=data_path,
            train='train.csv',
            validation = 'dev.csv',
            format='csv',
            fields=fields)
          
        print ('Building Vocabulary')
        self.CHAR.build_vocab(self.train, self.dev)
        self.WORDS.build_vocab(self.train, self.dev, vectors=GloVe(name='6B', dim=glove_size))

        print ('Creating Iterators')
        self.train_iter = PreprocessData.create_train_iterator(self.train, device, batch_size)
        self.dev_iter = PreprocessData.create_dev_iterator(self.dev, device, batch_size)
                    
            
    @staticmethod
    def create_train_iterator(train, device, batch_size):
        train_iter = data.BucketIterator(
            train,
            batch_size=batch_size,
            device=device,
            repeat=False,
            shuffle=True,
            sort_within_batch=True,
            sort_key=lambda x: len(x.context_words))
        
        return train_iter
    
    @staticmethod
    def create_dev_iterator(dev, device, batch_size):
        dev_iter = data.BucketIterator(
            dev,
            batch_size = batch_size,
            device=device,
            repeat=False,
            sort_within_batch=True,
            sort_key=lambda x: len(x.context_words))
        
        return dev_iter

In [6]:
create_checkpoint_paths('./data_checkpoints/')
create_checkpoint_paths('./model_checkpoints/')

In [11]:
_ = LoadData(data_path='../../../../data/squad v1.1/', checkpoint_path='./data_checkpoints/')


Flattening SQUAD 1.1 - Train
Completed: 87599 | Time: 0:25:09
Erronous Train Datapoints: 0

Flattening SQUAD Dev
Completed: 34726 | Time: 0:23:36
Erronous Dev Datapoints: 0


In [7]:
data_preprocessor = PreprocessData(data_path = './data_checkpoints/',
                                   glove_size = 100,
                                   batch_size = 120)

Loading CSV Data Into Torch Tabular Dataset
Building Vocabulary
Creating Iterators


In [8]:
class BidaF(nn.Module):
    def __init__(self, 
                 WORDS, 
                 CHAR,
                 char_embedding_size, # each character is represented by
                 char_conv_kernel_size, # width of convolution layer kernel
                 char_conv_channels_count, # no. of convolution layer channels
                 dropout=0.2,
                ): #
        super(BidaF, self).__init__()
        self.embedding_size = WORDS.vocab.vectors.shape[1] + char_conv_channels_count
        
        # 1. Word Embeddings
        self.word_embedding_layer = nn.Embedding.from_pretrained(WORDS.vocab.vectors, freeze=True)
        
        # 2. Char Embeddings
        self.char_embedding_layer = nn.Embedding(len(CHAR.vocab), char_embedding_size)
        self.convolution_layer = nn.Conv2d(1, char_conv_channels_count, (char_conv_kernel_size, char_embedding_size))
        nn.init.xavier_uniform_(self.convolution_layer.weight)
        self.dropout = nn.Dropout(p=dropout)
        
        # 3. Highway Network
        self.highway_linear1 = nn.Sequential(nn.Linear(self.embedding_size, self.embedding_size), nn.ReLU())
        self.highway_gate1 = nn.Sequential(nn.Linear(self.embedding_size, self.embedding_size), nn.Sigmoid())
        
        self.highway_linear2 = nn.Sequential(nn.Linear(self.embedding_size, self.embedding_size), nn.ReLU())
        self.highway_gate2 = nn.Sequential(nn.Linear(self.embedding_size, self.embedding_size), nn.Sigmoid())
        
        # 4. Embedding LSTM Layer
        self.embed_LSTM = nn.LSTM(input_size=self.embedding_size, 
                                  hidden_size=self.embedding_size, 
                                  num_layers=1,
                                  batch_first=True, 
                                  bidirectional=True)
        nn.init.xavier_normal_(self.embed_LSTM.weight_hh_l0)
        nn.init.xavier_normal_(self.embed_LSTM.weight_ih_l0)
        
        # 5. Attention Flow Layer
        self.w = torch.empty((6*self.embedding_size, 1), device=device)
        nn.init.xavier_normal_(self.w)
        
        # 6. Modelling Layer
        self.modelling_LSTM1 = nn.LSTM(input_size=8*self.embedding_size, 
                                       hidden_size=self.embedding_size,
                                       num_layers=1,
                                       batch_first=True,
                                       bidirectional=True)
        nn.init.xavier_normal_(self.modelling_LSTM1.weight_hh_l0)
        nn.init.xavier_normal_(self.modelling_LSTM1.weight_ih_l0)
        
        self.modelling_LSTM2 = nn.LSTM(input_size=2*self.embedding_size, 
                                       hidden_size=self.embedding_size,
                                       num_layers=1,
                                       batch_first=True,
                                       bidirectional=True)
        nn.init.xavier_normal_(self.modelling_LSTM2.weight_hh_l0)
        nn.init.xavier_normal_(self.modelling_LSTM2.weight_ih_l0)
        
        # 7. Output Layer
        self.p1_w = nn.Linear(10 * self.embedding_size, 1)
        nn.init.xavier_normal_(self.p1_w.weight)
        
        self.p2_w = nn.Linear(10 * self.embedding_size, 1)
        nn.init.xavier_normal_(self.p2_w.weight)
        
    def word_embeddings(self, x):
        x = self.word_embedding_layer(x)
        return x
    
    def character_embeddings(self, x):
        x = self.char_embedding_layer(x) # batch X words X char X char_emb
        _s = x.shape
        
        # Reshaping vector as we wish to run the character level embedding over each word in the batch
        x = x.view(_s[0]*_s[1], 1, _s[2], _s[3])  
        x = self.convolution_layer(x)
        x = nn.functional.relu(x).squeeze(3)
        x = nn.functional.max_pool1d(x, x.shape[2]).squeeze(2)
        x = self.dropout(x)
        
        # Shaping back to original batch_size, max_count_of_word_in_batch, char_conv_channels_count
        x = x.view(_s[0], _s[1], -1)
        
        return x

    
    def highway_network(self, x1, x2):
        x = torch.cat([x1, x2], -1)
        
        x = self.highway_linear1(x)
        t1 = self.highway_gate1(x)
        
        x = t1 * x + (1 - t1) * x
        
        x = self.highway_linear2(x)
        t2 = self.highway_gate2(x)
        
        x = t2 * x + (1 - t2) * x
        
        return x
    
    def lstm_embedding(self, x, x_lengths):
        x = torch.nn.utils.rnn.pack_padded_sequence(x, x_lengths, enforce_sorted=False, batch_first=True)
        x, _ = self.embed_LSTM(x)
        x, _ = torch.nn.utils.rnn.pad_packed_sequence(x, batch_first=True)
        x = self.dropout(x)
        
        return x
    
    def attention_flow_layer(self, c, q, c_len, q_len, batch_size):
        # Calculate the similarity matrix
        S = torch.empty((batch_size, c_len, q_len), device=device)
        for t in range(c_len):
            for j in range(q_len):
                _x = torch.cat([c[:, t, :], q[:, j, :], c[:, t, :] * q[:, j, :]], dim=-1)
                S[:, t, j] = torch.mm(_x, self.w).squeeze(1)
                
        # Calculating Context to Query Attention Matrix
        c2q_attn = torch.empty((batch_size, c_len, 2*self.embedding_size), device=device)
        A = F.softmax(S, dim=1)
        for t in range(c_len):
            c2q_attn[:, t, :] = (S[:, t, :].unsqueeze(2) * q).sum(1)
        
        # Calculating Query to Context Attention Matrix
        z, _ = torch.max(S ,dim=2)
        b = F.softmax(z, dim=1).unsqueeze(2)
        _h = (b * c2q_attn).sum(1).unsqueeze(1)
        q2c_attn = _h.repeat(1, c_len, 1)
        
        # Merging all Information
        G = torch.cat([c, c2q_attn, c * c2q_attn, c * q2c_attn], dim=-1)
        
        return G
    
    def modelling_layer(self, x):
        m1, _ = self.modelling_LSTM1(x)
        m1 = self.dropout(m1)
        
        m2, _ = self.modelling_LSTM2(m1)
        m2 = self.dropout(m2)
        
        return m1, m2
    
    def output_layer(self, G, m1, m2):
        
        p1 = self.p1_w(torch.cat([G, m1], dim=-1)).squeeze(2)
        p2 = self.p2_w(torch.cat([G, m2], dim=-1)).squeeze(2)
        return p1, p2
    
    def mask_output(self, p1, p2):  
        # This function replaced the pads with zeros to remove them from loss calculations
        max_size = p1.shape[1]
        mask = torch.tensor([[1.0] * lengths.item() + [0.0] * (max_size - lengths.item()) 
                             for lengths in train_data.context_words[1]])
        
        return mask * p1, mask * p2
        
    def forward(self, data):
        context_word_emb = self.word_embeddings(data.context_words[0])
        ques_word_emb = self.word_embeddings(data.question_words[0])
                                               
        context_char_emb = self.character_embeddings(data.context_char)
        ques_char_emb = self.character_embeddings(data.question_char)
        
        context_emb = self.highway_network(context_word_emb, context_char_emb)
        ques_emb = self.highway_network(ques_word_emb, ques_char_emb)
        
        context_emb = self.lstm_embedding(context_emb, data.context_words[1])
        ques_emb = self.lstm_embedding(ques_emb, data.question_words[1])
        
        G = self.attention_flow_layer(context_emb, ques_emb, 
                                      data.context_words[1].max().item(), data.question_words[1].max().item(),
                                      data.batch_size)
        
        m1, m2 = self.modelling_layer(G)
        p1, p2 = self.output_layer(G, m1, m2)
        
        # masking output before softmax to remove pads from output calculations
        # p1, p2 = self.mask_output(p1, p2)
      
        p1 = F.softmax(p1, dim=-1)
        p2 = F.softmax(p2, dim=-1)
        
        return p1, p2           

In [9]:
def clean_text(text):
    text = text.lower()
    text = [i for i in text if i.isalnum() or i.isspace()]
    return "".join(text)

In [10]:
def accuracy(context, predicted_p1, predicted_p2, actual_p1, actual_p2, method):
    acutal_ans =' '.join(context[actual_p1:actual_p2])
    predicted_ans = ' '.join(context[predicted_p1:predicted_p2])
    acutal_ans = clean_text(acutal_ans)
    predicted_ans = clean_text(predicted_ans)
#     print (acutal_ans)
#     print (predicted_ans)
    if method == 'exact_match':  
        return 1 if acutal_ans == predicted_ans else 0
    else:
        pred_tok = predicted_ans.split()
        actual_tok = acutal_ans.split()
        common = Counter(pred_tok) & Counter(actual_tok)
        count_common_words = sum(common.values())
        if count_common_words == 0:
            return 0.0
        precision = 1.0 * count_common_words / len(pred_tok) if len(pred_tok) > 0 else 0.0
        recall = 1.0 * count_common_words / len(actual_tok)
        f1 = (2 * precision * recall) / (precision + recall)
        return f1

In [11]:
def evaluate_batch_accuracy(data, vocab, p1, p2):
    p1, p2 = p1.argmax(1), p2.argmax(1)
    context_text = [[vocab.itos[word_idx.item()] for word_idx in context_idx] for context_idx in data.context_words[0]]
    
    exact_accuracy = [accuracy(context_text[i], 
                               p1[i].item(), p2[i].item(), 
                               data.start_idx[i].item(), data.end_idx[i].item(),
                               method='exact_match') 
                      for i in range(data.batch_size)]
    
    f1_accuracy = [accuracy(context_text[i], 
                            p1[i].item(), p2[i].item(), 
                            data.start_idx[i].item(), data.end_idx[i].item(),
                            method='f1') 
                   for i in range(data.batch_size)]
    
    return exact_accuracy, f1_accuracy

In [12]:
def evaluate_accuracy(data, model, vocab):
    model.eval()
    index, exact_accurancy, f1_accuracy = [], [], []
    for batch_data in iter(data):
        p1, p2 = model(batch_data)
        batch_exact_accuracy, batch_f1_accuracy = evaluate_batch_accuracy(batch_data, vocab, p1, p2)
        index += batch_data.id
        exact_accurancy += batch_exact_accuracy
        f1_accuracy += batch_f1_accuracy
        
        
    results = pd.DataFrame({'id': index, 'Exact': exact_accurancy, 'F1': f1_accuracy})
    return results, results.groupby('id')['Exact'].max().mean(), results.groupby('id')['F1'].max().mean()

In [13]:
def evaluate_loss(data, model):
    model.eval()
    total_loss = 0
    examples_count = 0.0
    for batch_data in iter(data):
        examples_count += batch_data.batch_size
        p1, p2 = model(batch_data)
        batch_loss = criterion(p1, batch_data.start_idx) + criterion(p2, batch_data.end_idx)
        total_loss += batch_loss.item()
        
        
    return total_loss/examples_count

In [14]:
def train(model, optimizer, criterion, path, epochs=200, epochs_log=10, model_checkpoint=5, pretrained_model=None):
    start_time = time.time()      
    for epoch in range(epochs):
        # SETTING MODEL IN TRAINING MODE
        model.train()
        
        epoch_loss = 0
        batch_num = 0.0
        exmaples_count = 0.0
        best_dev_acc_exact = -1.0
        best_dev_acc_f1 = -1.0
        
        for train_data in iter(data_preprocessor.train_iter):
            batch_num += 1.0
            exmaples_count += train_data.batch_size
            p1, p2 = model(train_data)
            optimizer.zero_grad()
            try:
                batch_loss = criterion(p1, train_data.start_idx) + criterion(p2, train_data.end_idx)
            except Exception as e:
                print (e)
                return (p1, p2, train_data)
                
            epoch_loss += batch_loss.item()
            batch_loss.backward()
            
            optimizer.step()
            
            time_delta = datetime.timedelta(seconds=np.round(time.time() - start_time, 0))
            sys.stdout.write(f'\rEpoch:{epoch} | Batch:{batch_num} | Time Running: {time_delta}')
            
            
        if epoch % epochs_log == 0:
#             print (epoch_loss, exmaples_count)
            triain_loss = epoch_loss/(exmaples_count)
            dev_loss = evaluate_loss(data_preprocessor.dev_iter, model)
            
            _, train_accuracy_exact, train_accuracy_f1 = evaluate_accuracy(data_preprocessor.train_iter, 
                                                                        model, 
                                                                        data_preprocessor.WORDS.vocab)
            _, dev_accuracy_exact, dev_accuracy_f1 = evaluate_accuracy(data_preprocessor.dev_iter, 
                                                                    model, 
                                                                    data_preprocessor.WORDS.vocab)
            
            print (f'\nTrain Loss:{triain_loss:.4f} Train Acc Exact:{train_accuracy_exact:.4f} Train Acc F1:{train_accuracy_f1:.4f}')
            print (f'Validation Loss :{dev_loss:.4f} Dev Acc Exact:{dev_accuracy_exact:.4f} Dev Acc F1:{dev_accuracy_f1:.4f}')
            
            print ('Test Prediction Results')
            predict_context = "He was speaking after figures showed that the country's economy shrank by 20.4% in April - the largest monthly contraction on record - as the country spent its first full month in lockdown."
            predict_ques = "By how much did the country's economy shrank"
            print (predition(predict_context, predict_ques, model, data_preprocessor))
            
            if dev_accuracy_f1 > best_dev_acc_f1:
                best_dev_acc_f1 = dev_accuracy_f1
                best_dev_acc_exact = dev_accuracy_exact
                best_model = copy.deepcopy(model)
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': epoch_loss,
                }, path + 'best_moodel.torch')
            
            
        if epoch%model_checkpoint == 0:   
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': epoch_loss,
            }, path + 'model.torch')

In [17]:
def predict_data(context_tokenized, question_tokenized, model, preprocessor):
    
    longest_context_word = max([len(w) for w in context_tokenized])
    longest_question_word = max([len(w) for w in question_tokenized])
    
    context_words = (torch.tensor([[preprocessor.WORDS.vocab[word.lower()] for word in context_tokenized]]),
                     torch.tensor([len(context_tokenized)]))
    
    question_words = (torch.tensor([[preprocessor.WORDS.vocab[word.lower()] for word in question_tokenized]]),
                     torch.tensor([len(question_tokenized)]))
    
    context_char = []
    for word in context_tokenized:
        _context_word = []
        for c_index in range(longest_context_word):
            if c_index < len(word):
                _context_word.append(preprocessor.CHAR.vocab[word[c_index]])
            else:
                _context_word.append(preprocessor.CHAR.vocab['<pad>'])
        context_char.append(_context_word)  
    context_char = torch.tensor([context_char])
    
    question_char = []
    for word in question_tokenized:
        _question_word = []
        for c_index in range(longest_question_word):
            if c_index < len(word):
                _question_word.append(preprocessor.CHAR.vocab[word[c_index]])
            else:
                _question_word.append(preprocessor.CHAR.vocab['<pad>'])
        question_char.append(_question_word)
    question_char = torch.tensor([question_char])
    
    predict_data.context_words = context_words
    predict_data.question_words = question_words
    predict_data.context_char = context_char
    predict_data.question_char = question_char
    predict_data.batch_size = 1
    
def predition(context, question, model, preprocessor):
    model.eval()
    context_tokenized = ptbtokenizer(context, context=True)
    question_tokenized = ptbtokenizer(question)
    
    predict_data(context_tokenized, question_tokenized, model, preprocessor)
    p1, p2 = model(predict_data)
    
    answer = " ".join(context_tokenized[p1.argmax(1): p2.argmax(1)])
    
    return answer, p1.argmax(1), p2.argmax(1)

In [18]:
PATH = './model_checkpoints/'

# Loading from Stored Path
model = BidaF(data_preprocessor.WORDS, 
              data_preprocessor.CHAR,
              char_embedding_size = 100,
              char_conv_kernel_size = 5,
              char_conv_channels_count = 100)

# optimizer = optim.Adam(model.parameters(), lr=0.001)
optimizer = optim.Adadelta(model.parameters(), lr=0.5, rho=0.999)
criterion = nn.CrossEntropyLoss()

if os.path.isfile(PATH + 'moodel.torch'):
    checkpoint = torch.load(PATH + 'moodel.torch')
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    print ('Loaded Trained Model')
    print (f'Trained for {epoch} Epochs, Achieved {loss} Training Loss')
    
_error = train(model=model, 
          optimizer=optimizer, 
          criterion=criterion, 
          path=PATH, 
          epochs=10, 
          epochs_log=1, 
          model_checkpoint=2, 
          pretrained_model=None)

Epoch:0 | Batch:133.0 | Time Running: 9:40:32

KeyboardInterrupt: 

In [ ]:
_p1, _p2, _data = _error[0], _error[1], _error[2]

In [133]:
_data.context_words[1]

tensor([102, 102, 102, 102, 102, 102, 102, 102, 102, 102, 102, 102, 102, 102,
        102, 102, 102, 102, 102, 102, 102, 102, 102, 102, 102, 102, 102, 102,
        102, 102, 102, 102, 102, 102, 102, 102, 102, 102, 101, 101, 101, 101,
        101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101,
        101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101,
        101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101,
        101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101,
        101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101,
        101, 101, 101, 101, 101, 101, 101, 101])

In [134]:
_data.end_idx

tensor([ 82,  57,  49,  27,  17,  85,  69,  94,  25,   9,  35,  61,  32,  14,
         51,  17, 100,  96,  88,  11,  93,  54,  14, 102,  82,  14,  34,  99,
          5,  38,  49,  88,  93,  62,  77,  46,  63,  14,  25,  92,   6,  31,
         80,  34,   4,  88,   1,  12,  17,  69,  19,   2,  70,  23,  83,   8,
         92,  31,  39,  17,  87,   7, 101,  90,  78,  97,   2,  49, 100, 100,
         19,  91,  25,  37,  80,  26,  51,  33,   6,   3, 100,  62,  78,  78,
         81,  41,   6,  50,  53, 100, 101,  66,  80,  90,  79,  93,  65,  69,
         93,   8,  89,  45,   2,  86,  12,  41,  49,  65,  54,  56,  73,  70,
         14,  25,  22,  94,  70,  22,  18,  19])

In [129]:
_p1.argmax(1)

tensor([ 13,   0,  68,  99,  51,   3,  28,  60,  72,  21,  10, 101,  94, 100,
         62,  98,   0,   1,  93,  97,  90,  47, 100,  20, 100,  50,  66,  98,
         86,   0,  43,  15,  40,  40,   1, 101,  28,  84,   6, 101,  69,   0,
         40,  14,  90, 101,  44,  95,  34,  89,  20,  37,  20,   6,  49,  89,
         92, 101,  44,  56,  37,  52,  66,  86,   0, 101,  59,   1,  67,   0,
          0,  97,   7,  12,  55,  20,   0,  20,  76,  98,  14,  77,  73,  83,
         32,  89,  13,  31,  69,  22,  94,  31,   0,  67,  83,  78,  25,   0,
          1,  31,  18,  82, 100,  14,   0,  93, 101,  37, 101,  88,  53,  61,
         33,  49,   9,  59,  41,  44,  94,   0])

In [130]:
_p2.argmax(1)

tensor([ 13,   0,  68,  99,  51,   3,  28,  60,  72,  21,  10, 101,  94, 100,
         62,  98,   0,   1,  93,  97,  90,  47, 100,  20, 100,  50,  66,  98,
         86,   0,  43,  15,  40,  40,   1, 101,  28,  84,   6, 101,  69,   0,
         40,  14,  90, 101,  44,  95,  34,  89,  20,  37,  20,   6,  49,  89,
         92, 101,  44,  56,  37,  52,  66,  86,   0, 101,  59,   1,  67,   0,
          0,  97,   7,  12,  55,  20,   0,  20,  76,  98,  14,  77,  73,  83,
         32,  89,  13,  31,  69,  22,  94,  31,   0,  67,  83,  78,  25,   0,
          1,  31,  18,  82, 100,  14,   0,  93, 101,  37, 101,  88,  53,  61,
         33,  49,   9,  59,  41,  44,  94,   0])

In [97]:
predict_context = "He was speaking after figures showed that the country's economy shrank by 20.4% in April - the largest monthly contraction on record - as the country spent its first full month in lockdown."
predict_ques = "By how much did the country's economy shrank"

predition(predict_context, predict_ques, model, data_preprocessor)

("He was speaking after figures showed that the country 's economy shrank by 20 . 4",
 tensor([0]),
 tensor([16]))